In [1]:
import pandas as pd
from urllib.parse import quote
from google.colab import files
import numpy as np

In [2]:
df= pd.read_csv('/content/miniseries.csv',encoding='utf-8')
df=df.loc[df['방송'].str.len()>1]
df

,Unnamed: 0,작품명,극본,연출,방송 기간,방송,방송사,유형,UHD,방송.1,시청,비고
0,3,국민 여러분!,한정훈,김정현,"2019.05.28, 2019.04.01",36 (18),KBS2,월화,NaN,NaN,(),8회 연장
1,10,어쩌다 발견한 하루,"인지혜, 송하영",김상협,"2019.10.02, 2019.11.21",32 (16),MBC,수목,NaN,NaN,(),사전제작
2,12,위대한 쇼,설준석,신용휘,"2019.10.15, 2019.8.26",16,tvN,월화,NaN,NaN,(),21회 조기종영
3,13,유령을 잡아라,"이영주, 소원",신윤섭,"2019.10.21, 2019.12.10",16,tvN,월화,NaN,NaN,(),사전제작
4,16,달리는 조사관,백정철,"김용수, 이창희","2019.10.31, 2019.9.18",14,OCN,수목,NaN,NaN,(),2회 조기종영
...,...,...,...,...,...,...,...,...,...,...,...,...
200,411,"어쩌다 마주친, 그대",백소연,"강수연, 이웅희","2023.6.20, 2023.5.1",16,KBS2,월화,NaN,NaN,(),사전제작
201,415,가슴이 뛴다,"정승주, 김하나","이민수, 이현석",2023.6.26,16,KBS2,월화,NaN,NaN,(),사전제작
202,416,기적의 형제,김지우,박찬홍,2023.6.28,16,JTBC,수목,NaN,NaN,(),사전제작
203,418,이로운 사기,한우주,이수현,"2023.7.18, 2023.5.29",16,tvN,월화,NaN,NaN,(),사전제작


In [3]:
drama_list = df['작품명'].tolist()

In [4]:
meta1=pd.DataFrame()

for name in drama_list:
  title = pd.DataFrame({"작품명":[name]})
  try:

    url = f"https://ko.wikipedia.org/wiki/{name}"
    encoded_url = quote(url, safe=':/_')
    tables = pd.read_html(encoded_url)

    # 수정 )) 이렇게 해야 _(드라마)로 넘어갈 수 있는데 제가 실수했습니다. 아래의 try는 굳이 바꿀 필요 없어서 냅뒀어요.
    for i in tables:
      if i.iloc[2,0] == '장르' or i.iloc[1,0] == '장르':
        table = i
        break

    table = table[0:].dropna().T
    table.columns = table.iloc[0]
    table = table[1:]

    desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

    for col in desired:
      if col not in table.columns:
        table[col] = np.nan

    table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

    table = pd.concat([title,table],axis=1)

        # 이전 이후 드라마
    for i in tables:
      try:
        if i.columns[0][1] == '이전 작품':
          df3 = i
          break
      except Exception:
        pass
    df3.columns = ['이전', '현재', '다음']
    table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

  except Exception:

    try:
      url = f"https://ko.wikipedia.org/wiki/{name}_(드라마)"
      encoded_url = quote(url, safe=':/_')
      tables = pd.read_html(encoded_url)
      table = tables[0]

      table = table[0:].dropna().T
      table.columns = table.iloc[0]
      table = table[1:]

      desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

      for col in desired:
        if col not in table.columns:
          table[col] = np.nan

      table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

      table = pd.concat([title,table],axis=1)

      # 이전 이후 드라마
      for i in tables:
        try:
          if i.columns[0][1] == '이전 작품':
            df3 = i
            break
        except Exception:
          pass
      df3.columns = ['이전', '현재', '다음']
      table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

    except Exception:
      table = title

  meta1=pd.concat([meta1,table],ignore_index=True) # concat 함수 내부 meta -> meta1


<ipython-input-4-9f2eea9248dd>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[col] = np.nan
<ipython-input-4-9f2eea9248dd>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[col] = np.nan
<ipython-input-4-9f2eea9248dd>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [7]:
meta1[meta1['장르'].isna()]["작품명"].unique()
## 코드를 약간 수정한 결과 에러리스트가 대폭 감소하였습니다.

array(['보좌관 2', '아이템', nan, '빙의', '빅이슈', '에이틴 / 에이틴 시즌 2', '나의 위험한 아내',
       '아무도 모른다', '악의 꽃', '멀리서 보는 푸른 봄', '이브', '치얼 업', '커튼 콜'],
      dtype=object)

In [8]:
# error_list = ['보좌관_2_-_세상을_움직이는_사람들','VIP_(드라마)','아이템_(드라마)','해치_(드라마)','빙의_(드라마)','빅이슈_(2019년_드라마)','어비스_(드라마)','바람이_분다_(드라마)','퍼퓸_(드라마)',
#               '레벨업_(드라마)','에이틴_(드라마)','에이틴_2','저스티스_(드라마)','나를_사랑한_스파이_(드라마)','카이로스_(드라마)','나의_위험한_아내_(대한민국의_드라마)','사생활_(드라마)','낮과_밤_(드라마)',
#               '블랙독_(드라마)','방법_(드라마)','포레스트_(드라마)','머니게임_(드라마)','아무도_모른다_(드라마)','본_어게인_(드라마)','저녁_같이_드실래요%3F_(드라마)','오_마이_베이비_(드라마)','출사표_(드라마)',
#               '악의_꽃_(2020년_드라마)','마우스_(드라마)','나빌레라_(드라마)','로스쿨_(드라마)','멀리서_보면_푸른_봄_(드라마)','홈타운_(드라마)','펜트하우스_(드라마)','연모_(드라마)','스폰서_(드라마)','크레이지_러브_(드라마)',
#               '인사이더_(드라마)','이브_(2022년_드라마)','치얼업_(드라마)','커튼콜_(드라마)','트롤리_(드라마)','오아시스_(드라마)','나쁜_엄마_(드라마)','패밀리_(드라마)']

# 줄어든 에러리스트입니다.
error_list = ['보좌관_2_-_세상을_움직이는_사람들','아이템_(드라마)','빙의_(드라마)','빅이슈_(2019년_드라마)','에이틴_(드라마)','에이틴_2','나의_위험한_아내_(대한민국의_드라마)','아무도_모른다_(드라마)',
              '악의_꽃_(2020년_드라마)','멀리서_보면_푸른_봄_(드라마)','이브_(2022년_드라마)','치얼업_(드라마)','커튼콜_(드라마)']


In [9]:
# 사실 에러리스트를 만든 이상 try구문은 필요 없습니다. 에러리스트는 손수 구한 정확한 단어들이어서, 오류가 생기지 않거든요.
# 그러나 이하에서는 코드를 더 안 건드리겠습니다. 맨 마지막 concat 안을 meta2로만 바꿨습니다.
meta2=pd.DataFrame()

for name in error_list:
  title = pd.DataFrame({"작품명":[name]})
  try:

    url = f"https://ko.wikipedia.org/wiki/{name}"
    encoded_url = quote(url, safe=':/_')
    tables = pd.read_html(encoded_url)
    table = tables[0]

    table = table[0:].dropna().T
    table.columns = table.iloc[0]
    table = table[1:]

    desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

    for col in desired:
      if col not in table.columns:
        table[col] = np.nan

    table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

    table = pd.concat([title,table],axis=1)

        # 이전 이후 드라마
    for i in tables:
      try:
        if i.columns[0][1] == '이전 작품':
          df3 = i
          break
      except Exception:
        pass
    df3.columns = ['이전', '현재', '다음']
    table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

  except Exception:

    try:
      url = f"https://ko.wikipedia.org/wiki/{name}_(드라마)"
      encoded_url = quote(url, safe=':/_')
      tables = pd.read_html(encoded_url)
      table=tables[0]

      table = table[0:].dropna().T
      table.columns = table.iloc[0]
      table = table[1:]

      desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

      for col in desired:
        if col not in table.columns:
          table[col] = np.nan

      table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

      table = pd.concat([title,table],axis=1)

            # 이전 이후 드라마
      for i in tables:
        try:
          if i.columns[0][1] == '이전 작품':
            df3 = i
            break
        except Exception:
          pass
      df3.columns = ['이전', '현재', '다음']
      table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

    except Exception:
      table = title

  meta2=pd.concat([meta2,table],ignore_index=True)


In [10]:
meta = pd.concat([meta1, meta2], ignore_index = True)

In [13]:
meta.to_csv('미니시리즈_메타데이터.csv', index=False, encoding='utf-8-sig')
files.download('미니시리즈_메타데이터.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>